In [1]:
import pandas as pd
import requests
import zipfile
import io
import os

In [2]:
data = pd.read_csv('data/mscz-files.csv')
print(data.shape)
data.head()


(1621519, 3)


,id,ref,path
0,6555648,/ipfs/QmbVaMHLD82LQsDyXSiZeQs7CCv7diZ1fEPQWUJY...,NaN
1,6315008,/ipfs/QmZDTiDxK8B97W8jYW4veosHNfmbFE785oX2fsKm...,NaN
2,6791936,/ipfs/QmRuaruxSRDJsJ4iG7wW1GPcb4nQmzJfcN3LzMP8...,NaN
3,6791936,/ipfs/QmRuaruxSRDJsJ4iG7wW1GPcb4nQmzJfcN3LzMP8...,NaN
4,6733568,/ipfs/QmZi1TC7VgA7iGKsnTF8FJyVZk7344Sxx22CpZAV...,NaN


In [3]:
def musicxmlify(file_path):
    return file_path.replace('.mscx','.musicxml').replace('mscx','sample')

def download(ref):
    return requests.get("https://dweb.link"+ref,stream=True)
    


def load_data(file):
    zip = zipfile.ZipFile(io.BytesIO(file.content))
    filename = [f for f in zip.namelist() if f.endswith('.mscx')][0]
    # check if file already exists
    if os.path.exists(musicxmlify('mscx/'+filename)):
        return False
    with open('./mscx/'+filename, 'wb') as f:
        f.write(zip.read(filename)) 
    return filename   

In [4]:
import subprocess

def convert(file):
    # get absolute path of file

    file_path = os.path.abspath('mscx/'+file)

    new_file_path = musicxmlify(file_path)

    command = subprocess.Popen([r'C:\Program Files\MuseScore 3\bin\MuseScore3.exe','-o',new_file_path,file_path],stdout=subprocess.PIPE)
    command.communicate()


In [5]:
def helper(arg):
    file = load_data(download(arg))
    if not file:
        return
    convert(file)
    x=already_downloaded()
    if x%10==0:
        print(x,end='\r',flush=True)

In [6]:
def clear(folder_name):
    # delete all files in folder
    filelist = os.listdir(folder_name)
    for f in filelist:
        os.remove(os.path.join(folder_name, f))

def already_downloaded():
    return len(os.listdir(musicxmlify('mscx')))

#clear('mscx')
#clear('musicxml')

In [7]:
from concurrent.futures import ThreadPoolExecutor,as_completed
from tqdm import tqdm

processes = []
n=len(data['ref'])


print('Downloading files...')
with ThreadPoolExecutor(max_workers=100) as executor:

    for url in data['ref'][:n]:
        processes.append(executor.submit(helper, url))
        

print('Converting to MusicXML')
print(f'Out of {n} files')
for task in as_completed(processes):
    try:
        pass
    except Exception as e:
        print(e)